In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier


In [ ]:
df_val = pd.read_csv("https://raw.githubusercontent.com/wvs2/data-hate/master/HateVal/prob_val_hateval.csv").dropna()
df_test = pd.read_csv("https://raw.githubusercontent.com/wvs2/data-hate/master/HateVal/prob_test_hateval.csv").dropna()

test_original = pd.read_csv("hateval2019_en_test.csv")

In [ ]:
test, class_test = df_test.drop(columns=["Unnamed: 0", "HS"]), df_test["HS"]
val, class_val = df_val.drop(columns=["Unnamed: 0", "HS"]), df_val["HS"]

In [ ]:
machines = ["SVM", "LR", "RF", "NB", "EXTRA", "KNN", "MLP", "CNN"]
features = ["CV", "TFIDF", "W2V", "GLOVE", "FAST"]
labels = ["0", "1"]
clfs = {}

for machine in machines:
  clfs[machine] = []
  for feature in features:
    if machine == "CNN" and feature == "TFIDF":
      feature = "TF"
    for label in labels:
      clfs[machine].append("{}-{}-{}".format(machine, feature, label))


clfs_by_features = {}
for feature in features:
  clfs_by_features[feature] = []
  for machine in machines:
    for label in labels:
      if machine == "CNN" and feature == "TFIDF":
        clfs_by_features[feature].append("CNN-TF-{}".format(label))
      else:
        clfs_by_features[feature].append("{}-{}-{}".format(machine, feature, label))

In [ ]:
for machine, clf in clfs.items():
  cols = clfs[machine]
  # Instancia de MLP
  val_vision = val[cols]    
  test_vision = test[cols]

  mlp = MLPClassifier(random_state=42, batch_size=20, verbose=100, activation='logistic', solver='lbfgs', max_iter=200)
  mlp.fit(val_vision, class_val)
  y_pred = mlp.predict(test_vision)
  print(machine)
  print("Accuracy: ",accuracy_score(class_test, y_pred))
  print("F1: ", f1_score(class_test, y_pred, average="macro"))
  print("")


SVM
Accuracy:  0.5136666666666667
F1:  0.4850679711454538



/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


LR
Accuracy:  0.515
F1:  0.47993359830309423

RF
Accuracy:  0.44966666666666666
F1:  0.37233809193060974



/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


NB
Accuracy:  0.5213333333333333
F1:  0.4976032362375007

EXTRA
Accuracy:  0.443
F1:  0.354097307501864



/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


KNN
Accuracy:  0.5313333333333333
F1:  0.5287058158909046

MLP
Accuracy:  0.5083333333333333
F1:  0.48195148232705287

CNN
Accuracy:  0.5193333333333333
F1:  0.493621930595068



In [ ]:
for feature, clfs in clfs_by_features.items():
  # for clf in clfs:
  cols = clfs
  # # Instancia de MLP
  val_vision = val[cols]    
  test_vision = test[cols]

  mlp = MLPClassifier(random_state=42, batch_size=20, verbose=100, activation='logistic', solver='lbfgs')
  mlp.fit(val_vision, class_val)
  y_pred = mlp.predict(test_vision)
  print(feature)
  print("Accuracy: ",accuracy_score(class_test, y_pred))
  print("F1: ", f1_score(class_test, y_pred, average="macro"))
  # print("")


CV
Accuracy:  0.44766666666666666
F1:  0.36419468305085767
TFIDF
Accuracy:  0.45166666666666666
F1:  0.3708867819297267
W2V
Accuracy:  0.6086666666666667
F1:  0.6065119259732965
GLOVE
Accuracy:  0.5976666666666667
F1:  0.5976430170173335
FAST
Accuracy:  0.5666666666666667
F1:  0.5610953080413192


In [ ]:
val_vision = val
test_vision = test

mlp = MLPClassifier(random_state=42, batch_size=20, verbose=100, activation='logistic', solver='lbfgs')
mlp.fit(val_vision, class_val)
y_pred = mlp.predict(test_vision)
# data_frame = pd.DataFrame(y_pred)
print(" TODOS ")
print("Accuracy: ",accuracy_score(class_test, y_pred))
print("F1: ", f1_score(class_test, y_pred, average="macro"))
# print("")

 TODOS 
Accuracy:  0.4736666666666667
F1:  0.40919832190871475


In [ ]:
index = 0
export_pred = {
    "i": [],
    "pred": []
}
for y in y_pred:
  export_pred["i"].append(test_original["id"][index])
  export_pred["pred"].append(y)
  index += 1
data_frame = pd.DataFrame(export_pred)
data_frame.to_csv("en_a.tsv", sep="\t")

In [ ]:
clfs = [
    "KNN-CV", "CNN-FAST", "RF-W2V", "LR-CV","RF-TFIDF", "NB-W2V", "KNN-FAST", "RF-GLOVE", "LR-GLOVE", 
    "SVM-CV", "SVM-GLOVE"
]

In [ ]:
cols = []
for clf in clfs:
  for i in range(0,2):
    cols.append("{}-{}".format(clf, i))

val_vision = val[cols]
test_vision = test[cols]
mlp = MLPClassifier(random_state=42, activation='logistic')
mlp.fit(val_vision, class_val)
y_pred = mlp.predict(test_vision)
print("=============== COMBINE ===============")
print("Accuracy: ",accuracy_score(class_test, y_pred))
print("F1: ", f1_score(class_test, y_pred, average="macro"))

=============== COMBINE ===============
Accuracy:  0.485
F1:  0.44084886742038065


In [ ]:
index = 0
export_pred = {
    "i": [],
    "pred": []
}
for y in y_pred:
  export_pred["i"].append(test_original["id"][index])
  export_pred["pred"].append(y)
  index += 1
data_frame = pd.DataFrame(export_pred)
data_frame.to_csv("en_proposed.tsv", sep="\t")